In [1]:
import geopandas as gp
import pandas as pd
from shapely.geometry import Polygon
from routingpy.routers import Valhalla
import numpy as np
import matplotlib.pyplot as plt

# Load environment variables
from dotenv import load_dotenv
import os
load_dotenv()

%matplotlib widget

In [2]:
# TODO: Get exact positions

df = pd.DataFrame(
    {
        "City": ["Oslo", "Bergen", "Trondheim", "Tromsø"],
        "LAT_VALUE": [59.94325569843703, 60.373620086371766, 63.42115156339022, 69.68334764603148],
        "LON_VALUE": [10.713583958247312, 5.357896567780437, 10.388075242832075, 18.986223899373528],
    }
)

gdf = gp.GeoDataFrame(
    df, geometry=gp.points_from_xy(df.LON_VALUE, df.LAT_VALUE), crs="EPSG:4326"
)

,City,LAT_VALUE,LON_VALUE,geometry
0,Oslo,59.943256,10.713584,POINT (10.71358 59.94326)
1,Bergen,60.373620,5.357897,POINT (5.35790 60.37362)
2,Trondheim,63.421152,10.388075,POINT (10.38808 63.42115)
3,Tromsø,69.683348,18.986224,POINT (18.98622 69.68335)


In [3]:
def valhalla_isochrone(client, gdf, time = [60], profile = "driving"):

    # Grab X and Y values in 4326
    gdf['LON_VALUE'] = gdf.to_crs(4326).geometry.x
    gdf['LAT_VALUE'] = gdf.to_crs(4326).geometry.y

    coordinates = gdf[['LON_VALUE', 'LAT_VALUE']].values.tolist()

    # Build a list of shapes
    isochrone_shapes = []

    if type(time) is not list:
        time = [time]

    # Use minutes as input, but the API requires seconds
    time_seconds = [60 * x for x in time]

    # Given the way that routingpy works, we need to iterate through the list of 
    # coordinate pairs, then iterate through the object returned and extract the 
    # isochrone geometries.  
    for c in coordinates:
        iso_request = client.isochrones(locations = c, profile = profile,
                                    intervals = time_seconds)

        for i in iso_request:
            iso_geom = Polygon(i.geometry)
            isochrone_shapes.append(iso_geom)

    # Here, we re-build the dataset but with isochrone geometries
    df_values = gdf.drop(columns = ['geometry', 'LON_VALUE', 'LAT_VALUE'])

    time_col = time * len(df_values)

    # We'll need to repeat the dataframe to account for multiple time intervals
    df_values_rep = pd.DataFrame(np.repeat(df_values.values, len(time_seconds), axis = 0))
    df_values_rep.columns = df_values.columns

    isochrone_gdf = gp.GeoDataFrame(
        data = df_values_rep,
        geometry = isochrone_shapes,
        crs = 4326
    )

    isochrone_gdf['time'] = time_col

    # We are sorting the dataframe in descending order of time to improve visualization
    # (the smallest isochrones should go on top, which means they are plotted last)
    isochrone_gdf = isochrone_gdf.sort_values('time', ascending = False)

    return(isochrone_gdf)


In [ ]:
# Fecth isochrones from VALHALLA and save
client = Valhalla(base_url="http://0.0.0.0:8002")
i = valhalla_isochrone(client, gdf, time =[30, 60, 90, 120], profile = "auto")

In [6]:
i.explore(column = "time")
